## PyMuPDF4LLM .md 전처리
- 공백 테이블 병합
- 병합셀 채워넣기
- 현재: 해당 테이블이 위치하는 페이지 리스트를 입력하여 병합된 표(df)를 얻음
- 주의: 컬럼을 잘못 읽어온 경우 예외처리, 한 페이지에 여러 표가 있는 경우

In [ ]:
import pymupdf4llm

# PDF data 추출
llama_reader = pymupdf4llm.LlamaMarkdownReader()
llama_docs = llama_reader.load_data(r"data\pdf_data\LH-25년1차청년매입임대입주자모집공고문(서울지역본부).pdf")


### 1) 컬럼 예외처리

In [ ]:
import pandas as pd
import re
from io import StringIO
import re
from llama_index.core.schema import Document

special_chars = ['•', '■', '※']

def fix_invalid_column_lines(md_text: str) -> str:
    lines = md_text.splitlines()
    new_lines = []
    i = 0

    while i < len(lines):
        line = lines[i]

        # 조건: | 2개 이상 + 특수기호 포함 + 다음줄 구분선 + 다다음 줄 존재
        if (
            line.count('|') >= 2 and 
            any(char in line for char in special_chars) and 
            i + 2 < len(lines)
        ):
            next_line = lines[i + 1].strip()
            next_next_line = lines[i + 2].strip()

            if re.fullmatch(r'\|?[-| ]+\|?', next_line):
                print("컬럼 순서 변경:", line)

                # 잘못된 줄 (| 제거) 먼저 올림
                new_lines.append(line.replace('|', '').strip())
                # 실제 컬럼명
                new_lines.append(next_next_line)
                # 구분선
                new_lines.append(next_line)

                i += 3
                continue

        # 그 외는 그대로
        new_lines.append(line)
        i += 1

    return '\n'.join(new_lines)

# llama_docs 각 문서에 적용
cleaned_docs = []
for doc in llama_docs:
    modified = fix_invalid_column_lines(doc.text)
    cleaned_docs.append(Document(text=modified))
cleaned_docs


In [ ]:
re.findall(r'(\|.*?\|\n(\|[-:]+[-|:]*\|\n)(\|.*?\|\n)+)', cleaned_docs[13].text, re.DOTALL)

### 2) 공백 테이블 전처리 (테이블 병합 및 병합셀 채우기)

In [ ]:
import re

def merge_pagetext(docs, page_list):
    merged_text = ""
    for page in page_list:
        text = docs[page-1].text
        merged_text += text
    merged_text = merged_text.replace('-----', '') #  페이지 구분 선

    # 페이지 넘버 문자열 제거
    for page_num in page_list:
        merged_text = merged_text.replace(f"- {page_num}", '')

    return merged_text

def is_table_separator(line):
    return re.match(r'^\s*\|?[:\-]+(?:\|[:\-]+)*\|?\s*$', line.strip())

def is_table_row(line):
    return re.match(r'^\s*\|.*\|\s*$', line.strip())

def is_ignore_line(line):
    # 페이지 나눔 같은 구분선: -----, =====, *** 등
    return re.match(r'^\s*[-=*]{3,}\s*$', line.strip())

def extract_combined_tables(text):
    lines = text.splitlines()
    tables = []
    current_table = []
    inside_table = False
    is_column = False

    for line in lines:
        if is_table_row(line) or is_table_separator(line):
            current_table.append(line.strip())
            inside_table = True
        elif is_ignore_line(line):
            # 페이지 구분 등은 무시하고 표 계속 이어붙임
            continue
        elif line.strip() == '':
            # 빈 줄은 무시 (표 끊지 않음)
            continue
        else:
            # 일반 텍스트: 표 끝으로 간주
            if inside_table and current_table:
                tables.append('\n'.join(current_table))
                current_table = []
                inside_table = False

    # 끝에 표가 남아 있다면 추가
    if current_table:
        tables.append('\n'.join(current_table))

    return tables

# 행 수 세는 함수 (헤더 제외, 구분선 제외)
def count_rows(md_table):
    lines = md_table.strip().splitlines()
    return max(0, len(lines) - 2)

# 마크다운 테이블 재구성 함수
def make_merged_table_md(merged_text):

    # 각 줄 단위로 분리
    lines = merged_text.strip().split('\n')

    # 첫 번째 컬럼만 헤더 공백값 전처리
    lines[0] = lines[0].replace('||', '|Col|')

    # 여기서 지우자 잘못 들어간 구분선
    

    # 결과를 저장할 리스트
    merged_table = []
    merged_table.extend(lines[:2])
    header_found = True
    for i in range(2, len(lines)):
        line = lines[i]
        # print(line)
        if '|---|' in line:
            merged_table.pop()
            line = ''

        if line == '':
            continue
        if re.match(r'^\|.*\|$', line.strip()):
            # 헤더가 아직 발견되지 않았다면 첫 헤더와 구분선만 추가
            if not header_found and re.match(r'^\|[^|]+\|[^|]+\|*', line):
                merged_table.append(line)
                header_found = True
            # 구분선은 무시 (두 번째 이후는)
            elif '---' in line:
                continue
            else:
                # 나머지는 데이터 행으로 처리
                merged_table.append(line)

    # 결과 출력 (마크다운 표 형태)
    return '\n'.join(merged_table)

# 마크다운 테이블 재구성 함수
def make_merged_table_df(merged_text):
    # table_blocks = re.findall(r'(\|.*?\|\n(\|[-:]+[-|:]*\|\n)(\|.*?\|\n)+)', merged_text, re.DOTALL)
    # first_block = table_blocks[0]
    # table_md = first_block[0]
    # rows = table_md.strip().split("\n")

    rows = merged_text.strip().split("\n")
    header = rows[0]  # 컬럼명 (첫 번째 행)
    column_list = [col.strip() for col in header.split("|") if col.strip()]
    data_rows = rows[2:]

    df = pd.read_csv(StringIO("\n".join([header] + data_rows)), sep="|", engine="python", skipinitialspace=True)
    df = df.iloc[:, 1:-1]  # 앞뒤 공백 컬럼 제거
    df.columns = column_list 
    df = df[0:].reset_index(drop=True)  # 데이터만 유지

    # ffill은 따로 해주기 (병합셀 심화 과정- 함수 안에서 하거나 반환한 테이블 전처리 따로)
    return df

table_df_list = []
# page_list = [9, 10, 11]
for page_list in extended_page_list:
    full_text = merge_pagetext(cleaned_docs, page_list)

    table_list = extract_combined_tables(full_text)

    # 행이 가장 많은 표 하나만 가져오기
    max_table = max(table_list, key=count_rows)

    # 표 재구성
    merged_table_md = make_merged_table_md(max_table)
    df = make_merged_table_df(merged_table_md)

    target_df = df.ffill(axis=1).ffill(axis=0)
    table_df_list.append(target_df)
# target_df



## Azure DI .md 전처리
- ■ bullet 예외 데이터 전처리 (제목에 부연 설명 있는 경우, 줄바꿈 안 된 경우 등)
- ■ 계층구조 삭제
- 공백 테이블 교체 (from PyMuPDF4LLM) -> (개발 중)
- html 테이블 -> 텍스트로 설명된 테이블 (from LLM) -> (함수화 및 병합 필요)

In [ ]:
# markdown 파일 읽기
with open("document_result.md", "r", encoding="utf-8") as file:
    azure_md = file.read()

### 1) ■ bullet 예외 데이터 전처리, 계층구조 삭제

In [3]:
import re

# 페이지 분할: 페이지 번호와 내용을 그룹으로 가져오기
split_pages = re.split(r'<!--\s*PageNumber="([^"]*)"\s*-->', azure_md)

# 불필요한 처음 항목 제거 (페이지 번호 앞 텍스트)
if split_pages and not split_pages[0].strip():
    split_pages = split_pages[1:]

# 페이지 목록을 (페이지번호, 내용) 튜플로 묶기
page_pairs = list(zip(split_pages[::2], split_pages[1::2]))

# 전처리 및 페이지 재조합
restructured_pages = []

for page_number, content in page_pairs:
    lines = content.splitlines()
    new_lines = []

    for line in lines:
        if "■" in line:
            # 동작1: '#' 등 마크다운 계층 구조 제거
            cleaned_line = re.sub(r'^[#>\-\*\s]+', '', line)
            # 동작2: ■ 앞에 공백이 있다면 \n■ 처리
            cleaned_line = re.sub(r'\s*■', r'\n■', cleaned_line)
            # 동작3: ':'가 있다면 \n: 처리
            cleaned_line = re.sub(r'\s* :\s*', r'\n:', cleaned_line)
            # 동작4: ") " → ")\n"
            cleaned_line = re.sub(r'\)\s+', r')\n', cleaned_line)
            new_lines.append(cleaned_line)
        else:
            new_lines.append(line)

    # 전처리된 페이지 내용 조합
    modified_content = '\n'.join(new_lines)
    # page_comment = f'<!-- PageNumber="{page_number.strip()}" -->'
    restructured_pages.append(f"{page_number.strip()}")

# 최종 마크다운 문서
# final_markdown = '\n\n<!-- PageBreak -->\n\n'.join(restructured_pages)


# 결과 출력
print("\n--- Final Markdown ---\n")
print(restructured_pages)


NameError: name 'azure_md' is not defined

### 2) 공백 테이블 교체 (from PyMuPDF4LLM)

In [ ]:
# Azure DI .md에서 연장표 index 찾기 (반례 가능성 有)
import re
from bs4 import BeautifulSoup

def split_pages(markdown_text):
    pattern = r'<!-- PageNumber="- (\d+) -" -->\s*<!-- PageBreak -->'
    parts = re.split(pattern, markdown_text)
    pages = []
    for i in range(1, len(parts), 2):
        page_num = int(parts[i]) + 1
        content = parts[i+1].strip()
        pages.append((page_num, content))
    return pages

def is_table_only(html_text):
    soup = BeautifulSoup(html_text, 'html.parser')
    text = soup.get_text().strip()
    # 태그 중 table 관련 태그만 있는지 확인
    allowed_tags = {'table', 'tr', 'td', 'th'}
    all_tags = {tag.name for tag in soup.find_all()}
    return (text == '') and all_tags.issubset(allowed_tags)

def detect_table_transition(pages):
    transitions = []
    page_unit = []
    for i in range(len(pages) - 1):
        
        curr_page_num, curr_content = pages[i]
        next_page_num, next_content = pages[i + 1]

        if curr_content.strip().endswith('</table>') and next_content.strip().startswith('<table>'): # 현 페이지가 </table>로 끝나고 다음 페이지가 <table>로 시작할 때
            if curr_page_num in page_unit: # 이미 페이지가 page_unit에 있다면
                if curr_content.strip().startswith('<table>') and curr_content.strip().endswith('</table>'):
                    if len(curr_content.split('</table>')) > 2: # 현재 페이지 안에 표가 2개 이상
                        transitions.append(page_unit) # 현재 페이지까지 저장된 unit 저장
                        page_unit = []
                        page_unit.append(curr_page_num) # 현재 페이지부터 다시 카운트
                        page_unit.append(next_page_num)
                    else:
                        page_unit.append(next_page_num) # 한 페이지 전체가 표. 다음 페이지만 저장
                else:
                    transitions.append(page_unit)
                    page_unit = []
                    page_unit.append(curr_page_num)
            else:
                if page_unit:
                    transitions.append(page_unit)
                    page_unit = []
                page_unit.append(curr_page_num)
                page_unit.append(next_page_num)
                
    transitions.append(page_unit)
    return transitions

def merge_transitions(transitions, pages_dict):
    if not transitions:
        return []

    merged = []
    current_group = transitions[0]

    for next_pair in transitions[1:]:
        prev_last = current_group[-1]
        next_first = next_pair[0]

        if prev_last == next_first:
            # 중간 페이지가 table-only이면 병합
            if is_table_only(pages_dict[prev_last]):
                current_group.append(next_pair[1])
            else:
                merged.append(current_group)
                current_group = next_pair
        else:
            merged.append(current_group)
            current_group = next_pair

    merged.append(current_group)
    return merged

def process_markdown_for_table_groups(markdown_text):
    pages = split_pages(markdown_text)
    pages_dict = dict(pages)
    transitions = detect_table_transition(pages)
    merged_groups = merge_transitions(transitions, pages_dict)
    return merged_groups

# with open("document_result.md", "r", encoding="utf-8") as file:
#     azure_md = file.read()

extended_page_list = process_markdown_for_table_groups(azure_md)

print(extended_page_list)  # [[3, 4, 5], [7, 8], [10, 11]]


In [ ]:
# 표 교체하는 코드 (PyMuPDF4LLM 전처리 과정 이후 사용)
# 완성 후 Final Markdown 저장
import re

# 전처리 및 페이지 재조합
pattern = re.compile(r'<table[\s\S]*?</table>', re.IGNORECASE)

for i in range(len(extended_page_list)):
    page_list = extended_page_list[i]
    df = table_df_list[i]
    new_html_table = df.to_html(index=False, escape=False)

    # 첫 페이지
    first_page = page_list[0]
    page_md = restructured_pages[first_page-1]
    matches = list(pattern.finditer(page_md))
    matched_table = matches[-1].group()
    new_page_md = page_md.replace(matched_table, new_html_table)
    restructured_pages[first_page-1] = new_page_md

    # 마지막 페이지 + 나머지
    for i in range(1, len(page_list)):
        page_md = restructured_pages[page_list[i]-1]
        matches = list(pattern.finditer(page_md))
        matched_table = matches[0].group()
        new_page_md = page_md.replace(matched_table, '')
        restructured_pages[page_list[i]-1] = new_page_md

final_md = '\n'.join(restructured_pages)

with open("output.md", "w", encoding="utf-8") as f:
    f.write(final_md)

### 3) html 테이블 -> 텍스트로 설명된 테이블 (from LLM) -> (함수화 및 병합 필요)

In [ ]:
# Jupyter나 VS Code에서 실행 시 사용할 수 있도록 수정한 버전

import os
from openai import AzureOpenAI
from dotenv import load_dotenv

def process_file(input_file_path, output_file_path=None):
    """
    Process an input file through Azure AI to convert HTML tables to text.
    """
    load_dotenv()
    endpoint = os.getenv("ENDPOINT_URL")
    deployment = os.getenv("DEPLOYMENT_NAME")
    subscription_key = os.getenv("AZURE_OPENAI_KEY")
    
    if not all([endpoint, deployment, subscription_key]):
        raise ValueError("환경변수 누락: ENDPOINT_URL, DEPLOYMENT_NAME, AZURE_OPENAI_KEY를 확인하세요.")
    
    client = AzureOpenAI(
        azure_endpoint=endpoint,
        api_key=subscription_key,
        api_version="2024-05-01-preview",
    )
    
    with open(input_file_path, 'r', encoding='utf-8') as file:
        input_content = file.read()
    
    chat_prompt = [
        {
            "role": "system",
            "content": [
                {
                    "type": "text",
                    "text": "너는 HTML 테이블을 읽고 자연스러운 서술형 텍스트로 변환하는 텍스트 변환 엔진이다.\n\n입력 데이터는 일반 텍스트와 HTML 코드가 혼합된 문서이며, 이 중 HTML 테이블(`<table>`) 형식으로 작성된 표만을 감지하여 사람이 읽기 쉬운 **자연스러운 텍스트**로 변환하라. 표 외의 일반 텍스트는 **절대로 변경하지 않는다**. \n\n출력된 텍스트는 아래의 기준을 모두 따라야 한다:\n\n1. 표의 계층 구조, 제목, 셀의 관계를 모두 파악하여 자연어로 기술한다.\n2. 셀이 병합된 경우 (`rowspan`, `colspan`)에는 의미적으로 내용을 통합하여 풀어서 설명한다.\n3. 표 안에 또 다른 표가 중첩되어 있는 경우에도 각 표를 계층적으로 처리하고, 문맥상 자연스럽게 연결되도록 한다.\n4. 빈 칸이 있는 경우, 내용을 유추하지 않고 \"(빈칸)\" 또는 \"해당 없음\" 등으로 명확하게 표기한다.\n5. 항목 간 구분은 \"■\", \"1.\", \"-\" 등을 사용하여 명확히 구분하고, 계층적으로 정리한다.\n6. 결과 텍스트는 반드시 문맥상 자연스럽고 일관되게 연결되어야 하며, 원래 문서의 흐름과 연결되도록 이어져야 한다.\n7. HTML 태그가 아닌 일반 텍스트 영역은 절대로 수정하거나 재구성하지 않는다.\n8. 결과는 마크다운 문서로 사용 가능한 수준의 가독성을 갖춰야 하며, 표를 설명하는 문장은 공식 문서나 계약서 스타일처럼 명료하고 단정하게 작성한다.\n\n예외나 애매한 구조가 있어도 최대한 의미를 보존하여 사람이 이해할 수 있도록 직관적으로 설명하라.\n\n입력 형식 예시:\n(본문 텍스트)\n<table>...</table>\n(본문 텍스트 계속)\n\n출력 형식 예시:\n(본문 텍스트)\n■ 항목명  \n- 내용1  \n- 내용2  \n이제 아래에 입력된 문서 내 HTML 테이블을 위 기준에 따라 서술형 텍스트로 변환하라."
                }
            ]
        },
        {
            "role": "user",
            "content": [
                {
                    "type": "text",
                    "text": input_content
                }
            ]
        }
    ]
    
    completion = client.chat.completions.create(
        model=deployment,
        messages=chat_prompt,
        max_tokens=1500,
        temperature=0.7,
        top_p=0.95,
        frequency_penalty=0,
        presence_penalty=0,
        stream=False
    )
    
    processed_text = completion.choices[0].message.content
    
    if not output_file_path:
        base, _ = os.path.splitext(input_file_path)
        output_file_path = f"{base}_processed.txt"
    
    with open(output_file_path, 'w', encoding='utf-8') as file:
        file.write(processed_text)
    
    print(f"✅ 변환 완료: {output_file_path}")
    return processed_text


#=======================================================================

# 🔽 여기서 input/output 경로 지정해서 실행
input_path = "document_result.md"  # ← 실제 파일 경로로 수정
output_path = "result.txt"  # 또는 "결과저장파일.txt"

process_file(input_path, output_path)